In [ ]:
# TODO: Katya
# Fig S2-S3-S4, 4

**Structure:**

1. Description of NeuralDataset class and how it works. How to download datasets from drive. How to create your own class for own dataset.

2. Example of how to download and work with all datasets used in study. Description of datasets and table of difference in parameters.

3. Building figures from paper and some details about them.

In [ ]:
# Gallego: Not all files are uploaded here, since the full size of dataset is 9 Gb. 
# describe shufflings and show
# 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from utils.datasets import (
    NeuralDataset,
    GraspingSuresh,
    ReachingGallego,
    ReachingChurchland,
    ReachingKalidindi,
    BehaviouralMante,
)

In [ ]:
from utils.color_palettes import bg_gray_color, jpca_palette_journ1, GR1, RB1, YP1, YS1, blue_yellow_cmap, BG1
from utils.color_palettes import journ_color_dict, alt_colors
from utils.data_processing import load_h5_file

import numpy as np
from utils.color_palettes import blue_yellow_cmap
from utils.utils import subtract_cc_mean, soft_normalize
from utils.utils import shuffle_data, compute_curvature, pca_rotation
import jPCA
from jPCA.util import plot_projections
from scipy.optimize import curve_fit
from scipy.stats import pearsonr
from tqdm import tqdm
# from utils.utils import fit_running_wave


from datasets_config import * # TODO

In [ ]:
PATH_DRAFT = 'images/draft_panels/'
figSN_dir = 'figSN/'

all_names = ['churhc', 'grasp', 'lfp', 'khalid', 'pfc']
all_params = [church_params, grasp_params, lfp_params, khalid_params, pfc_params]
all_path = [church, grasp, lfp, khaild, pfc]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

for nam, params, dpath in zip(all_names, all_params, all_path):
    d_dict = load_h5_file(DATA_PATH, dpath)    
    datas = d_dict["data"]
    go_cue = d_dict["align_cue_idx"]
    
    if nam == 'grasp':
        datas = d_dict["data"][:, 220:300, :]
        go_cue = go_cue - 220

    # create objects
    fig = plt.figure(figsize=(13, 6), constrained_layout=True)
    gs = GridSpec(2, 4, figure=fig) 

    # create sub plots as grid
    ax0 = fig.add_subplot(gs[:, 0])

    ax1 = fig.add_subplot(gs[0, 1])
    ax2 = fig.add_subplot(gs[0, 2])
    ax3 = fig.add_subplot(gs[0, 3])

    ax4 = fig.add_subplot(gs[1, 1:3])
    ax5 = fig.add_subplot(gs[1, 3])

    # 0 - peth
    plot_peth(datas, go_cue, ax0, fs=params['fs'], sub_mean=False)
    ax0.set_xlabel('Time, ms', fontsize=12)
    ax0.set_title('PETH', fontsize=14)

    # 4 - curvature 
    plot_curvature(datas, go_cue, ax4, fs=params['fs'], sub_mean=False)
    ax4.set_xlabel('Time, ms', fontsize=12)
    ax4.set_title('Curvature of trajectories', fontsize=14)

    # 1 - jPCA
    build_jPCA(datas, ax1, params['ts'], params['te'], c_siz=params['c_size'], arr_siz=params['arr_size'],
                cmap=params['cmap'])
    # ax4.set_aspect('equal')
    for axis in ['top', 'bottom', 'left', 'right']:
        ax1.spines[axis].set_linewidth(0.5)
    ax1.set_xticks([])
    ax1.set_yticks([])
    # ax1.set_xlabel('jPC1, a.u.', fontsize=9)
    # ax1.set_ylabel('jPC2, a.u.', fontsize=9)
    ax1.set_title('jPCA plane', fontsize=14)

    # 2 - PCA
    build_pca(datas, ax2, params['ts'], params['te'], c_siz=params['c_size_pca'], arr_siz=params['arr_size_pca'],
            cmap=params['cmap'])
    # ax5.set_aspect('equal')
    for axis in ['top', 'bottom', 'left', 'right']:
        ax2.spines[axis].set_linewidth(0.5)
    ax2.set_xticks([])
    ax2.set_yticks([])
    # ax2.set_xlabel('projection onto PC1, a.u.', fontsize=10)
    # ax2.set_ylabel('projection onto PC2, a.u.', fontsize=10)
    ax2.set_title('PCA plane', fontsize=14)

    # 3 - jPCA of our fitted model
    t = np.arange(datas.shape[1])
    r2 = plot_R2_fitting(datas, t, ax3, params['ts'], params['te'], c_siz=params['c_size'], arr_siz=params['arr_size'],
                        cmap=params['cmap'], pfc=params['pfc'], khalid=params['khalid'])
    for axis in ['top', 'bottom', 'left', 'right']:
        ax3.spines[axis].set_linewidth(0.5)
    ax3.set_xticks([])
    ax3.set_yticks([])
    # ax3.set_xlabel('projection onto jPC1, a.u.', fontsize=10)
    # ax3.set_ylabel('projection onto jPC2, a.u.', fontsize=10)
    ax3.set_title('jPCA plane (running wave model)', fontsize=14)

    # 5 - R2 fitting distribution
    ax5.hist(np.array(r2).ravel(), bins=params['bins'], color=journ_color_dict['blue'],
              edgecolor='white', linewidth=0.7)
    ax5.axvline(np.nanmean(r2), c=journ_color_dict['red'],
                label='mean={}'.format(round(np.nanmean(r2), 2)), ls='--', alpha=0.6) 
    ax5.yaxis.tick_right()
    ax5.set_xlim([0., 1.009])
    for axis in ['top', 'bottom', 'left', 'right']:
        ax5.spines[axis].set_linewidth(0.5)

    import matplotlib.ticker as ticker
    ax5.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
    ax5.yaxis.set_major_locator(ticker.MultipleLocator(40))
    ax5.legend()
    ax5.grid(alpha=0.3)
    ax5.set_facecolor(bg_gray_color) 
    ax5.set_title('Distribution of R2', fontsize=14)
    ax5.yaxis.set_major_locator(ticker.MultipleLocator(params['hist_space']))
    ax5.yaxis.tick_right()
    
    plt.savefig(PATH_DRAFT + figSN_dir + nam + '.pdf', dpi=300, format='pdf',
                bbox_inches='tight', pad_inches=0.0)
    plt.tight_layout()
    plt.show()

| Article & Authors                                                                                                                 | Smoothing Sigma (ms)   | Frequency Sampling (sec)     | Original Smoothing Sigma (ms)             |
|-----------------------------------------------------------------------------------------------------------------------------------|------------------------|------------------------------|-------------------------------------------|
| "Neural Population Dynamics During Reaching", Churchland et al.                                                                   | same as original paper | 0.01 (1 time point = 10 ms)  | 20 / 24 ms S.D. (for diff datasets)       |
| "Neural Population Dynamics in Motor Cortex are Different for Reach and Grasp", Suresh et al.                                     | 70                     | 0.01 (1 time point = 10 ms)  | 25 ms S.D.                                |
| "Local field potentials reflect cortical population dynamics in a region-specific and frequency-dependent manner", Gallego et al. | 50                     | 0.001 (1 time point = 30 ms) | bins of size 30 ms                        |
| "Rotational dynamics in motor cortex are consistent with a feedback controller", Kalidindi et al.                                 | same as original paper | -                            | 30 ms S.D.                                |
| "Context-dependent Computation by Recurrent Dynamics in Prefrontal Cortex", Mante et al.                                          | 20                     | 0.001 (1 time point = 1 ms)  | 12.5 ms and 50 ms (for plots and fitting) |